### 1. Let us start by importing all the necessary libraries which will be required in our analysis

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('treebank')
nltk.download('punkt')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### 2. Now 

In [3]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [4]:
# Splitting into train and validation
random.seed(1234)
train_set, validation_set = train_test_split(wsj,test_size=0.05)

print(len(train_set))
print(len(validation_set))

3718
196


In [5]:
train_set_words = [i for j in train_set for i in j]

In [6]:
len(train_set_words)

95617

In [7]:
unique_words_set = set(train_set_words)

In [8]:
len(unique_words_set)

12810

In [9]:
tokens_in_train = [i[0] for i in train_set_words]

In [10]:
unique_tokens_train = set(tokens_in_train)

In [11]:
len(tokens_in_train)

95617

In [12]:
len(unique_tokens_train)

12081

In [13]:
tags_train = [i[1] for i in train_set_words]

In [14]:
len(tags_train)

95617

In [15]:
unique_tags_train = set(tags_train)

In [16]:
len(unique_tags_train)

12

In [17]:
print(unique_tags_train)

{'NOUN', 'NUM', 'CONJ', '.', 'PRON', 'DET', 'PRT', 'ADP', 'ADV', 'X', 'VERB', 'ADJ'}


### Emission Probabilities

In [18]:
# compute word given tag: Emission Probability
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_set_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [19]:
# examples

# large
print("\n", "large")
print(word_given_tag('large', 'ADJ'))
print(word_given_tag('large', 'VERB'))
print(word_given_tag('large', 'NOUN'), "\n")

# will
print("\n", "will")
print(word_given_tag('will', 'VERB'))
print(word_given_tag('will', 'NOUN'))

# book
print("\n", "book")
print(word_given_tag('book', 'NOUN'))
print(word_given_tag('book', 'VERB'))

# He
print("\n", "He")
print(word_given_tag('He', 'NOUN'))
print(word_given_tag('He', 'PRON'))

# one
print("\n", "one")
print(word_given_tag('one', 'NOUN'))
print(word_given_tag('one', 'NUM'))
print(word_given_tag('one', 'ADP'))


 large
(29, 6064)
(0, 12849)
(0, 27490) 


 will
(259, 12849)
(1, 27490)

 book
(7, 27490)
(0, 12849)

 He
(0, 27490)
(69, 2587)

 one
(9, 27490)
(116, 3386)
(0, 9350)


In [20]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_set_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [21]:
# examples
print(t2_given_t1(t2='NOUN', t1='ADJ'))
print(t2_given_t1('NOUN', 'DET'))
print(t2_given_t1('NOUN', 'VERB'))
print(t2_given_t1('.', 'NOUN'))
print(t2_given_t1('VERB', 'NOUN'))

(4244, 6064)
(5275, 8244)
(1409, 12849)
(6595, 27490)
(4038, 27490)


In [22]:
#Please note P(tag|start) is same as P(tag|'.')
print(t2_given_t1('DET', '.'))
print(t2_given_t1('VERB', '.'))
print(t2_given_t1('NOUN', '.'))

(1920, 11148)
(991, 11148)
(2491, 11148)


In [23]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(unique_tags_train), len(unique_tags_train)), dtype='float32')
for i, t1 in enumerate(list(unique_tags_train)):
    for j, t2 in enumerate(list(unique_tags_train)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

tags_matrix

array([[2.65223712e-01, 9.63986944e-03, 4.25609313e-02, 2.39905417e-01,
        4.87449998e-03, 1.30229173e-02, 4.39432524e-02, 1.75773010e-01,
        1.71335023e-02, 2.91014910e-02, 1.46889776e-01, 1.19316112e-02],
       [3.52923810e-01, 1.84288248e-01, 1.35853514e-02, 1.16952159e-01,
        1.47666864e-03, 3.83933843e-03, 2.74660364e-02, 3.51447128e-02,
        2.95333727e-03, 2.10868284e-01, 1.83106903e-02, 3.21913771e-02],
       [3.49397600e-01, 4.21686731e-02, 4.63392033e-04, 3.47544029e-02,
        5.74606098e-02, 1.19091749e-01, 4.63392027e-03, 5.28266914e-02,
        5.51436506e-02, 7.41427252e-03, 1.58016682e-01, 1.18628360e-01],
       [2.23448157e-01, 8.11804831e-02, 5.81270196e-02, 9.25726593e-02,
        6.51237890e-02, 1.72228202e-01, 2.42195907e-03, 9.10477191e-02,
        5.29242903e-02, 2.75385715e-02, 8.88948664e-02, 4.44025844e-02],
       [2.10668728e-01, 7.73096271e-03, 5.02512557e-03, 3.98144573e-02,
        7.34441448e-03, 9.27715469e-03, 1.31426360e-02, 2.35

In [24]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(unique_tags_train), index=list(unique_tags_train))
tags_df

,NOUN,NUM,CONJ,.,PRON,DET,PRT,ADP,ADV,X,VERB,ADJ
NOUN,0.265224,0.009640,0.042561,0.239905,0.004874,0.013023,0.043943,0.175773,0.017134,0.029101,0.146890,0.011932
NUM,0.352924,0.184288,0.013585,0.116952,0.001477,0.003839,0.027466,0.035145,0.002953,0.210868,0.018311,0.032191
CONJ,0.349398,0.042169,0.000463,0.034754,0.057461,0.119092,0.004634,0.052827,0.055144,0.007414,0.158017,0.118628
.,0.223448,0.081180,0.058127,0.092573,0.065124,0.172228,0.002422,0.091048,0.052924,0.027539,0.088895,0.044403
PRON,0.210669,0.007731,0.005025,0.039814,0.007344,0.009277,0.013143,0.023579,0.034016,0.093545,0.480479,0.075377
DET,0.639859,0.022319,0.000485,0.017346,0.003518,0.005580,0.000243,0.008976,0.012615,0.045245,0.039423,0.204391
PRT,0.247877,0.056826,0.002286,0.043762,0.018942,0.101241,0.001633,0.020575,0.010124,0.013717,0.398759,0.084259
ADP,0.324706,0.062995,0.000856,0.039679,0.068342,0.323209,0.001497,0.017005,0.013476,0.034332,0.008663,0.105241
ADV,0.032280,0.032612,0.006988,0.137438,0.015641,0.068552,0.014975,0.117471,0.078536,0.022629,0.343760,0.129118
X,0.062799,0.002869,0.010520,0.164010,0.054511,0.054192,0.182977,0.145521,0.026458,0.074275,0.204335,0.017533


In [25]:
tags_df.loc['.', :]

NOUN    0.223448
NUM     0.081180
CONJ    0.058127
.       0.092573
PRON    0.065124
DET     0.172228
PRT     0.002422
ADP     0.091048
ADV     0.052924
X       0.027539
VERB    0.088895
ADJ     0.044403
Name: ., dtype: float32

In [26]:
len(train_set_words)

95617

In [64]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_set_words):
    state=[]
    map_word_state = {}
    T = list(set([pair[1] for pair in train_bag]))
    V = list(set([pair[0] for pair in train_bag]))
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        map_word_state[word] = state_max
        state.append(state_max)
        
    return list(zip(words,state))

In [28]:
# list of tagged words
validation_run_base = [tup for sent in validation_set for tup in sent]

# list of untagged words
validation_tagged_words = [tup[0] for sent in validation_set for tup in sent]
print(len(validation_set))
print(len(validation_tagged_words))

196
5059


In [29]:
%%time
# tagging the test sentences
start = time.time()
validation_Viterbi1 = Viterbi(validation_tagged_words)
end = time.time()
difference = end-start

CPU times: user 10min 46s, sys: 89.5 ms, total: 10min 46s
Wall time: 10min 46s


In [30]:
# accuracy
check = [i for i, j in zip(validation_Viterbi1, validation_run_base) if i == j] 
accuracy = len(check)/len(validation_Viterbi1)

print(accuracy)

0.9345720498122159


In [31]:
incorrect_tagged_cases = [[validation_run_base[i-1],j] for i, j in enumerate(zip(validation_Viterbi1, validation_run_base)) if j[0]!=j[1]]
print(incorrect_tagged_cases)

[[('regulated', 'VERB'), (('*-134', 'NOUN'), ('*-134', 'X'))], [('which', 'DET'), (('*T*-218', 'NOUN'), ('*T*-218', 'X'))], [('*T*-218', 'X'), (('reports', 'NOUN'), ('reports', 'VERB'))], [("'s", 'PRT'), (('floral', 'NOUN'), ('floral', 'ADJ'))], [('voters', 'NOUN'), (('disapprove', 'NOUN'), ('disapprove', 'VERB'))], [('had', 'VERB'), (('no', 'DET'), ('no', 'ADV'))], [('a', 'DET'), (('minimal', 'NOUN'), ('minimal', 'ADJ'))], [('$', '.'), (('7.2', 'NOUN'), ('7.2', 'NUM'))], [('.', '.'), (('Some', 'DET'), ('Some', 'ADV'))], [('to', 'PRT'), (('test', 'NOUN'), ('test', 'VERB'))], [('.', '.'), (('Most', 'ADJ'), ('Most', 'ADV'))], [('and', 'CONJ'), (('manufacturing', 'NOUN'), ('manufacturing', 'VERB'))], [('goverment', 'NOUN'), (('counts', 'NOUN'), ('counts', 'VERB'))], [('Dodge', 'NOUN'), (('counts', 'NOUN'), ('counts', 'VERB'))], [('a', 'DET'), (('first-time', 'NOUN'), ('first-time', 'ADJ'))], [('program', 'NOUN'), (('that', 'ADP'), ('that', 'DET'))], [('that', 'DET'), (('*T*-99', 'NOUN'), 

In [32]:
%%time
## Testing
test_cases_shared = 'Android is a mobile operating system developed by Google. Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013. Google and Twitter made a deal in 2015 that gave Google access to Twitter\'s firehose. Twitter is an online news and social networking service on which users post and interact with messages known as tweets. Before entering politics, Donald Trump was a domineering businessman and a television personality. The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years. This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe. Show me the cheapest round trips from Dallas to Atlanta. I would like to see flights from Denver to Philadelphia. Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco. NASA invited social media users to experience the launch of ICESAT-2 Satellite.'
test_words = word_tokenize(test_cases_shared)

tagged_seq = Viterbi(test_words)

CPU times: user 23.4 s, sys: 12 ms, total: 23.4 s
Wall time: 23.4 s


In [33]:
print(tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

In [117]:
## Modification -1: all unknown words are replaced by Noun (Noun being the most common tag)

def Viterbi_modf1(test_words, train_bag = train_set_words):
    tagged_seq = Viterbi(test_words)
    V = list(set([pair[0] for pair in train_bag]))
    
    words = [pair[0] for pair in tagged_seq]
    Viterbi_tags = [pair[1] for pair in tagged_seq]
    
    for key, word in enumerate(words):
        if word not in V:
            Viterbi_tags[key] = 'NOUN'
            
    
    return list(zip(words, Viterbi_tags))

In [118]:
%%time
validation_Viterbimodf1 = Viterbi_modf1(validation_tagged_words)

CPU times: user 10min 40s, sys: 66 ms, total: 10min 40s
Wall time: 10min 40s


In [119]:
check_modf1 = [i for i, j in zip(validation_Viterbimodf1, validation_run_base) if i == j] 
accuracy = len(check_modf1)/len(validation_Viterbimodf1)
accuracy

0.9345720498122159

In [116]:
tagged_seq2_test = Viterbi_modf1(test_words)

In [86]:
validation_run_base_test

[('The', 'DET'),
 ('futures', 'NOUN'),
 ('industry', 'NOUN'),
 ('is', 'VERB'),
 ('regulated', 'VERB'),
 ('*-134', 'X'),
 ('by', 'ADP'),
 ('the', 'DET'),
 ('Commodity', 'NOUN'),
 ('Futures', 'NOUN'),
 ('Trading', 'NOUN'),
 ('Commission', 'NOUN'),
 (',', '.'),
 ('which', 'DET'),
 ('*T*-218', 'X'),
 ('reports', 'VERB'),
 ('to', 'PRT'),
 ('the', 'DET'),
 ('Agriculture', 'NOUN'),
 ('committees', 'NOUN'),
 ('in', 'ADP'),
 ('both', 'DET'),
 ('houses', 'NOUN'),
 ('.', '.'),
 ('Citizens', 'NOUN'),
 ('in', 'ADP'),
 ('Peninsula', 'NOUN'),
 (',', '.'),
 ('Ohio', 'NOUN'),
 (',', '.'),
 ('*-1', 'X'),
 ('upset', 'VERB'),
 ('over', 'ADP'),
 ('changes', 'NOUN'),
 ('to', 'PRT'),
 ('a', 'DET'),
 ('bridge', 'NOUN'),
 (',', '.'),
 ('negotiated', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 (':', '.'),
 ('The', 'DET'),
 ('bottom', 'NOUN'),
 ('half', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('railing', 'NOUN'),
 ('will', 'VERB'),
 ('be', 'VERB'),
 ('type', 'NOUN'),
 ('F', 'NOUN'),
 (',', '.'),
 ('while', 'A

In [102]:
validation_run_base

[('The', 'DET'),
 ('futures', 'NOUN'),
 ('industry', 'NOUN'),
 ('is', 'VERB'),
 ('regulated', 'VERB'),
 ('*-134', 'X'),
 ('by', 'ADP'),
 ('the', 'DET'),
 ('Commodity', 'NOUN'),
 ('Futures', 'NOUN'),
 ('Trading', 'NOUN'),
 ('Commission', 'NOUN'),
 (',', '.'),
 ('which', 'DET'),
 ('*T*-218', 'X'),
 ('reports', 'VERB'),
 ('to', 'PRT'),
 ('the', 'DET'),
 ('Agriculture', 'NOUN'),
 ('committees', 'NOUN'),
 ('in', 'ADP'),
 ('both', 'DET'),
 ('houses', 'NOUN'),
 ('.', '.'),
 ('Citizens', 'NOUN'),
 ('in', 'ADP'),
 ('Peninsula', 'NOUN'),
 (',', '.'),
 ('Ohio', 'NOUN'),
 (',', '.'),
 ('*-1', 'X'),
 ('upset', 'VERB'),
 ('over', 'ADP'),
 ('changes', 'NOUN'),
 ('to', 'PRT'),
 ('a', 'DET'),
 ('bridge', 'NOUN'),
 (',', '.'),
 ('negotiated', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 (':', '.'),
 ('The', 'DET'),
 ('bottom', 'NOUN'),
 ('half', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('railing', 'NOUN'),
 ('will', 'VERB'),
 ('be', 'VERB'),
 ('type', 'NOUN'),
 ('F', 'NOUN'),
 (',', '.'),
 ('while', 'A

In [103]:
validation_Viterbi1

[('The', 'DET'),
 ('futures', 'NOUN'),
 ('industry', 'NOUN'),
 ('is', 'VERB'),
 ('regulated', 'VERB'),
 ('*-134', 'NOUN'),
 ('by', 'ADP'),
 ('the', 'DET'),
 ('Commodity', 'NOUN'),
 ('Futures', 'NOUN'),
 ('Trading', 'NOUN'),
 ('Commission', 'NOUN'),
 (',', '.'),
 ('which', 'DET'),
 ('*T*-218', 'NOUN'),
 ('reports', 'NOUN'),
 ('to', 'PRT'),
 ('the', 'DET'),
 ('Agriculture', 'NOUN'),
 ('committees', 'NOUN'),
 ('in', 'ADP'),
 ('both', 'DET'),
 ('houses', 'NOUN'),
 ('.', '.'),
 ('Citizens', 'NOUN'),
 ('in', 'ADP'),
 ('Peninsula', 'NOUN'),
 (',', '.'),
 ('Ohio', 'NOUN'),
 (',', '.'),
 ('*-1', 'X'),
 ('upset', 'VERB'),
 ('over', 'ADP'),
 ('changes', 'NOUN'),
 ('to', 'PRT'),
 ('a', 'DET'),
 ('bridge', 'NOUN'),
 (',', '.'),
 ('negotiated', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 (':', '.'),
 ('The', 'DET'),
 ('bottom', 'NOUN'),
 ('half', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('railing', 'NOUN'),
 ('will', 'VERB'),
 ('be', 'VERB'),
 ('type', 'NOUN'),
 ('F', 'NOUN'),
 (',', '.'),
 ('whil

In [106]:
incorrect_tagged_cases = [(j) for i, j in enumerate(zip(validation_Viterbi1, validation_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[(('*-134', 'NOUN'), ('*-134', 'X')),
 (('*T*-218', 'NOUN'), ('*T*-218', 'X')),
 (('reports', 'NOUN'), ('reports', 'VERB')),
 (('floral', 'NOUN'), ('floral', 'ADJ')),
 (('disapprove', 'NOUN'), ('disapprove', 'VERB')),
 (('no', 'DET'), ('no', 'ADV')),
 (('minimal', 'NOUN'), ('minimal', 'ADJ')),
 (('7.2', 'NOUN'), ('7.2', 'NUM')),
 (('Some', 'DET'), ('Some', 'ADV')),
 (('test', 'NOUN'), ('test', 'VERB')),
 (('Most', 'ADJ'), ('Most', 'ADV')),
 (('manufacturing', 'NOUN'), ('manufacturing', 'VERB')),
 (('counts', 'NOUN'), ('counts', 'VERB')),
 (('counts', 'NOUN'), ('counts', 'VERB')),
 (('first-time', 'NOUN'), ('first-time', 'ADJ')),
 (('that', 'ADP'), ('that', 'DET')),
 (('*T*-99', 'NOUN'), ('*T*-99', 'X')),
 (('expunged', 'NOUN'), ('expunged', 'VERB')),
 (('which', 'DET'), ('which', 'ADP')),
 (('earlier', 'ADV'), ('earlier', 'ADJ')),
 (('agreed', 'VERB'), ('agreed', 'NOUN')),
 (('supply', 'NOUN'), ('supply', 'VERB')),
 (('more', 'ADV'), ('more', 'ADJ')),
 (('sank', 'NOUN'), ('sank', 'VERB

In [112]:
pd.Series(np.array([i[1][1] for i in incorrect_tagged_cases])).value_counts()

VERB    99
ADJ     87
NUM     32
NOUN    32
ADV     29
X       20
DET     18
ADP      8
PRT      3
.        2
CONJ     1
dtype: int64